In [1]:
import numpy as np
from numpy import linalg as LA
from numpy.linalg import matrix_power as MP

In [2]:
import random
import scipy
from scipy.spatial.distance import cdist
from scipy.io import savemat, loadmat

In [3]:
import networkx as nx
import sys

In [4]:
import tensorflow as tf
from keras.callbacks import EarlyStopping

In [5]:
print(f"TF Version: {tf.__version__}")

TF Version: 2.14.0


In [6]:
X_train = np.array(loadmat('./Dataset/X_train.mat')['X'])
N, T = X_train.shape

print(N, T)

32 744


In [7]:
dt = np.zeros([T - 1, T])

for i in range(T - 1):
    dt[i, i] = -1
    dt[i, i - 1] = 1

d = dt.T
d = np.concatenate((d, np.zeros([T, 1])), axis = 1) 
d[T - 1, T - 1] = -1
d = d.T
d[T - 1, T - 1] = 1 
d[-1, -2] = 1
d[0, -1] = 0
print(d)

d = tf.convert_to_tensor(d, dtype = "float32")

[[-1.  0.  0. ...  0.  0.  0.]
 [ 1. -1.  0. ...  0.  0.  0.]
 [ 0.  1. -1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -1.  0.  0.]
 [ 0.  0.  0. ...  1. -1.  0.]
 [ 0.  0.  0. ...  0.  1.  1.]]


In [8]:
Mul = np.eye(N) - np.ones([N, N])
Mul = tf.convert_to_tensor(Mul, dtype = "float32")

In [9]:
def initial_graph(X_training, N, top_values):
    distance_matrix = cdist(X_training, X_training, metric = 'euclidean')
    
    # top_values = 6
    sorted_indices = distance_matrix.argsort(1)
    TAdj = np.zeros([N, N])
    
    for i in range(N):
        for j in range(1, top_values):
            k = sorted_indices[i][j]
            TAdj[i][k] = 1
    
    TAdj = (TAdj + TAdj.T) / 2
    TAdj = np.where(TAdj > 0.1, 1, 0)
    
    Laplacian_init = np.diag(np.sum(TAdj, 1)) - TAdj
    
    return Laplacian_init

In [10]:
coeffs = 0
psi_cost = 0
Laplacian = 0
A_transform_cost = 0

model_pred = []
loss_model = []
learned_graph = []
mse_known_list = []
mse_unknown_list = []

coeffs_list = np.load("./Outputs/cost_func_coeffs.npy")

In [11]:
# A - Target, B - Predicted
def costfunc(A, B):
    global coeffs
    global X_train
    global psi_cost
    global Laplacian
    global A_transform_cost

    B_tmp = tf.squeeze(B)
    data_fidelity = (tf.norm(tf.multiply(psi_cost, (X_train - B))) ** 2) * coeffs[0]
    smoothness = tf.linalg.trace(tf.transpose(B_tmp) @ Laplacian @ B_tmp @ A_transform_cost) * coeffs[1]
    L_cost = (tf.norm(Laplacian) ** 2) * coeffs[2]
    Z_cost = (tf.norm(A_transform_cost) ** 2) * coeffs[3]
    
    tmp = (smoothness) + (data_fidelity) + (L_cost) + (Z_cost)
    
    global loss_model
    loss_model.append(tmp)
    
    return tmp
    
def costfunc1(A, B):
    global X_train
    global psi_cost
    
    data_fidelity = (tf.norm(tf.multiply(psi_cost, (X_train - B))) ** 2)
    return (data_fidelity)

def costfunc2(A, B):
    global Laplacian
    global A_transform_cost
    
    B_tmp = tf.squeeze(B)
    smoothness = tf.linalg.trace(tf.transpose(B_tmp) @ Laplacian @ B_tmp @ A_transform_cost)
    
    return (smoothness)

def costfunc3(A, B):
    global Laplacian
    
    L_cost = (tf.norm(Laplacian) ** 2)
    
    return (L_cost)

def costfunc4(A, B):
    global A_transform_cost
    
    Z_cost = (tf.norm(A_transform_cost) ** 2)
    
    return (Z_cost)

In [12]:
def mse_unknown(A, B):
    
    global psi_cost
    psi = psi_cost
    
    psid = tf.convert_to_tensor(np.ones([N, T], dtype = float) - psi, dtype = "float32")
    mse_unk = (LA.norm(np.multiply(psid, (X_train - B))) ** 2) / np.sum(psid.numpy().flatten())
    
    global mse_unknown_list
    mse_unknown_list.append(mse_unk)
    
    return mse_unk

def mse_known(A, B):
    
    global psi_cost
    psi = psi_cost
    
    mse_kn = (LA.norm(np.multiply(psi, (X_train - B))) ** 2) / np.sum(psi.numpy().flatten())
    
    global model_pred
    model_pred.append([A.numpy(), B.numpy()])
    
    global mse_known_list
    mse_known_list.append(mse_kn)
    
    return mse_kn

In [13]:
callb = EarlyStopping(monitor = 'loss', mode = 'min', min_delta = 0.0001, verbose = 1, patience = 10)

def scheduler(epoch, lr):
    if epoch < 40:
        return lr
    else:
        return lr / lr_decay_factor

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [14]:
class ProposedModel(tf.keras.Model):
    
    def __init__(self, L_init, reg_init, loops_init, d_init, p_init, alpha_init, beta_init, threshold_in, **kwargs):
        super(ProposedModel, self).__init__(**kwargs)
        
        self.p1_init = tf.constant(p_init[0], shape = (1, 1), dtype = "float32")
        self.p1 = tf.Variable(initial_value = self.p1_init, trainable = True, name = "p1")
        
        self.p2_init = tf.constant(p_init[1], shape = (1, 1), dtype = "float32")
        self.p2 = tf.Variable(initial_value = self.p2_init, trainable = True, name = "p2")
        
        self.L_in = L_init
        self.preg = reg_init
        self.loops = loops_init
        self.d_int = d_init
        self.alp = alpha_init
        self.bet = beta_init
        self.threshold_init = threshold_in
        
    def data_inpainting(self, psi, Y, Lk, AAT):
        loop_b, reg = self.loops[1], self.preg

        Xk = tf.zeros_like(Y)
        Zk = -(tf.multiply(psi, Xk) - Y + (2 * reg * (Lk @ Xk @ AAT)))
        
        for i in range(loop_b):

            fdx_xk = tf.multiply(psi, Xk) - Y + (2 * reg * (Lk @ Xk @ AAT))
            fdx_zk = tf.multiply(psi, Zk) - Y + (2 * reg * (Lk @ Zk @ AAT))
            
            tau = tf.linalg.trace(tf.transpose(fdx_xk) @ Zk) / tf.linalg.trace(tf.transpose((Y + fdx_zk)) @ Zk)
                        
            Xk_1 = Xk - (tau * Zk)

            fdx_xk_1 = tf.multiply(psi, Xk_1) - Y + (2 * reg * (Lk @ Xk_1 @ AAT))
            gamma = (tf.norm(fdx_xk_1) ** 2) / (tf.norm(fdx_xk) ** 2)

            Zk_1 = (gamma * Zk) - fdx_xk_1

            Xk = Xk_1
            Zk = Zk_1
        
        return Xk_1
    
    def graph_learning(self, X, Lk, AAT):
        loop_c, alpha, beta = self.loops[2], self.alp, self.bet
        
        X_AAT_XT = tf.matmul(X, tf.matmul(AAT, tf.transpose(X)))
        
        for i in range(loop_c):
            fdL_Lk = coeffs[1] * X_AAT_XT + (beta * Lk)

            ###### This is required
            fdL_Lk = fdL_Lk - tf.reshape(tf.reduce_mean(fdL_Lk, 1), [N, 1])
            fdL_Lk = (fdL_Lk + tf.transpose(fdL_Lk)) / 2
            ######
            
            Lk_1 = Lk - (alpha * fdL_Lk)
            
            ######
            Adj_k1 = tf.multiply(Mul, Lk_1)
            Adj_k1 = tf.nn.relu(Adj_k1)
            Lk_1 = tf.linalg.diag(tf.math.reduce_sum(Adj_k1, 1)) - Adj_k1
            ######
            
            Lk = Lk_1
        return Lk
    
    def call(self, y):

        psi = y[0,:,T:]
        global psi_cost
        psi_cost = psi
        
        psi = tf.convert_to_tensor(psi, dtype = "float32")
        
        X_in = y[0,:,:T]
        X_in = psi * tf.convert_to_tensor(X_in, dtype = "float32")
        Y = X_in
        
        d = self.d_int
        LT = tf.matmul(d, tf.transpose(d))
        LT = tf.convert_to_tensor(LT, dtype = "float32")
        
        A_transform = tf.eye(T) + (self.p1 * MP(LT, 1)) + (self.p2 * MP(LT, 2))
        AAT = A_transform
        
        global A_transform_cost
        A_transform_cost = AAT
        
        loop_a = self.loops[0]
        threshold = self.threshold_init
        Lk = self.L_in
        
        for i in range(loop_a):
            
            Xk_1 = self.data_inpainting(psi, Y, Lk, AAT)
            
            Xk_1_gt = X_in + tf.multiply((tf.ones(psi.shape) - psi), Xk_1)
            
            Lk_1 = self.graph_learning(Xk_1_gt, Lk, AAT)
            
            Adj = tf.linalg.diag(tf.linalg.diag_part(Lk_1)) - Lk_1 # Gives adjacency
            
            n = tf.math.count_nonzero(Adj, dtype="float32")
            mean = tf.math.reduce_sum(Adj) / n
            Adj = tf.math.subtract(Adj, (threshold * mean)) # Ensures the sparsity of learned graph
            
            Adj = tf.nn.relu(Adj)
            # Adj = tf.math.sign(Adj)
            
            Lk_1 = tf.linalg.diag(tf.math.reduce_sum(Adj, 1)) - Adj
            
            Lk = Lk_1
        
            global Laplacian
            Laplacian = Lk
        
        Xk_1 = self.data_inpainting(psi, Y, Lk, AAT)
        
        global learned_graph
        learned_graph.append(Lk.numpy())
        
        return tf.reshape(Xk_1, [1, N, T])

In [15]:
Lap_init = initial_graph(X_train, N, 10)

loop_a = 5
loop_b = 20 # Data Inpainting
loop_c = 10 # Graph Learning

reg_init = 4.0e-6

alpha_init = 1.0
beta_init = 1.0e-4

threshold_init = 0.3

no_of_psi = 10
# sensing_ratio = np.arange(1.0, 81.0, 1.0) / 100.0
sensing_ratio = np.array([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5])
# sensing_ratio = [0.1]

p_init = [10.0, 10.0]

lr_decay_factor = 1.04

all_psi = []
all_vars = []

In [16]:
for i_sen, rem in enumerate(sensing_ratio):
    
    print(f"\n\n###########################################################################################")
    print(f"################################### Sensing Ratio: {rem * 100}% ###################################")
    print(f"###########################################################################################\n\n")
    
    if i_sen == 0:
        M = int(rem * T)
        X_train_missing = np.zeros([no_of_psi, N, T])
        X_train_concatenated = np.zeros([no_of_psi, N, 2*T])
    
        for i in range(no_of_psi):
            psi_k = np.array([0] * (N * M) + [1] * (N * (T - M)))
            np.random.shuffle(psi_k)
            psi_k = psi_k.reshape([N, T])
            all_psi.append(psi_k)
            X_train_missing[i, : , : ] = X_train * psi_k
            X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)

    else:
        previous_psi = all_psi[-no_of_psi : ]
        X_train_missing = np.zeros([no_of_psi, N, T])
        X_train_concatenated = np.zeros([no_of_psi, N, 2 * T])
        
        rem_diff = int(N * (rem - sensing_ratio[i_sen - 1]) * T)
        
        for i in range(no_of_psi):
            psi_k_tmp = np.array(previous_psi[i]).flatten()
            ones_idx = np.where(psi_k_tmp == 1)
            
            ones_to_zero_idx = np.random.choice(ones_idx[0], size = rem_diff)
            psi_k_tmp[ones_to_zero_idx] = 0
            psi_k = psi_k_tmp
            
            psi_k = psi_k.reshape([N, T])
            all_psi.append(psi_k)
            X_train_missing[i, : , : ] = X_train * psi_k
            X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
    
    coeffs = coeffs_list[i_sen, : ]
    
    sgd_optimizer = tf.keras.optimizers.SGD(learning_rate = 4.0e-3, momentum = 0.001)
    
    our_model = ProposedModel(Lap_init, reg_init, [loop_a, loop_b, loop_c], 
                              d, p_init, alpha_init, beta_init, threshold_init, name = 'our')
    our_model.compile(optimizer = sgd_optimizer, loss = costfunc,
                      metrics = [mse_known, mse_unknown], run_eagerly=True)
    
    our_model.fit(X_train_concatenated, X_train_missing, epochs = 30, 
                  callbacks = [lr_scheduler], batch_size = 1)
    
    var = our_model.variables
    var_lst = []
    for v in var:
        var_lst.append(v.numpy()[0, 0])
    print(f"\nFilter Coefficients:", var_lst)
    all_vars.append(var_lst)



###########################################################################################
################################### Sensing Ratio: 1.0% ###################################
###########################################################################################


Epoch 1/30
10/10 [==============================] - 22s 2s/step - loss: 8.8008 - mse_known: 1.2718e-06 - mse_unknown: 0.2060 - lr: 0.0040
Epoch 2/30
10/10 [==============================] - 19s 2s/step - loss: 8.7221 - mse_known: 1.2627e-06 - mse_unknown: 0.2060 - lr: 0.0040
Epoch 3/30
10/10 [==============================] - 20s 2s/step - loss: 8.6442 - mse_known: 1.2527e-06 - mse_unknown: 0.2059 - lr: 0.0040
Epoch 4/30
10/10 [==============================] - 20s 2s/step - loss: 8.5670 - mse_known: 1.2439e-06 - mse_unknown: 0.2059 - lr: 0.0040
Epoch 5/30
10/10 [==============================] - 21s 2s/step - loss: 8.4903 - mse_known: 1.2330e-06 - mse_unknown: 0.2059 - lr: 0.0040
Epoch 6/30
10/10 [============


Filter Coefficients: [9.559109, 8.480312]


###########################################################################################
################################### Sensing Ratio: 5.0% ###################################
###########################################################################################


Epoch 1/30
10/10 [==============================] - 23s 2s/step - loss: 8.7949 - mse_known: 1.2162e-06 - mse_unknown: 0.2520 - lr: 0.0040
Epoch 2/30
10/10 [==============================] - 21s 2s/step - loss: 8.7163 - mse_known: 1.2061e-06 - mse_unknown: 0.2519 - lr: 0.0040
Epoch 3/30
10/10 [==============================] - 24s 2s/step - loss: 8.6384 - mse_known: 1.1972e-06 - mse_unknown: 0.2518 - lr: 0.0040
Epoch 4/30
10/10 [==============================] - 21s 2s/step - loss: 8.5612 - mse_known: 1.1869e-06 - mse_unknown: 0.2518 - lr: 0.0040
Epoch 5/30
10/10 [==============================] - 20s 2s/step - loss: 8.4847 - mse_known: 1.1786e-06 - mse_unknown: 0.2518 


Filter Coefficients: [9.559249, 8.480834]


###########################################################################################
################################### Sensing Ratio: 10.0% ###################################
###########################################################################################


Epoch 1/30
10/10 [==============================] - 27s 3s/step - loss: 8.7871 - mse_known: 1.1215e-06 - mse_unknown: 0.2543 - lr: 0.0040
Epoch 2/30
10/10 [==============================] - 25s 2s/step - loss: 8.7085 - mse_known: 1.1114e-06 - mse_unknown: 0.2543 - lr: 0.0040
Epoch 3/30
10/10 [==============================] - 26s 3s/step - loss: 8.6306 - mse_known: 1.1013e-06 - mse_unknown: 0.2542 - lr: 0.0040
Epoch 4/30
10/10 [==============================] - 24s 2s/step - loss: 8.5535 - mse_known: 1.0919e-06 - mse_unknown: 0.2542 - lr: 0.0040
Epoch 5/30
10/10 [==============================] - 22s 2s/step - loss: 8.4770 - mse_known: 1.0822e-06 - mse_unknown: 0.2541


Filter Coefficients: [9.559454, 8.481509]


###########################################################################################
################################### Sensing Ratio: 20.0% ###################################
###########################################################################################


Epoch 1/30
10/10 [==============================] - 23s 2s/step - loss: 8.7733 - mse_known: 9.4385e-07 - mse_unknown: 0.2730 - lr: 0.0040
Epoch 2/30
10/10 [==============================] - 26s 3s/step - loss: 8.6949 - mse_known: 9.3772e-07 - mse_unknown: 0.2729 - lr: 0.0040
Epoch 3/30
10/10 [==============================] - 29s 3s/step - loss: 8.6172 - mse_known: 9.2969e-07 - mse_unknown: 0.2728 - lr: 0.0040
Epoch 4/30
10/10 [==============================] - 25s 2s/step - loss: 8.5401 - mse_known: 9.2175e-07 - mse_unknown: 0.2727 - lr: 0.0040
Epoch 5/30
10/10 [==============================] - 25s 2s/step - loss: 8.4638 - mse_known: 9.1647e-07 - mse_unknown: 0.2726


Filter Coefficients: [9.559808, 8.482651]


###########################################################################################
################################### Sensing Ratio: 30.0% ###################################
###########################################################################################


Epoch 1/30
10/10 [==============================] - 25s 3s/step - loss: 9.3611 - mse_known: 9.6294e-07 - mse_unknown: 0.3293 - lr: 0.0040
Epoch 2/30
10/10 [==============================] - 22s 2s/step - loss: 9.2786 - mse_known: 9.5519e-07 - mse_unknown: 0.3290 - lr: 0.0040
Epoch 3/30
10/10 [==============================] - 24s 2s/step - loss: 9.1967 - mse_known: 9.4721e-07 - mse_unknown: 0.3288 - lr: 0.0040
Epoch 4/30
10/10 [==============================] - 24s 2s/step - loss: 9.1155 - mse_known: 9.3923e-07 - mse_unknown: 0.3285 - lr: 0.0040
Epoch 5/30
10/10 [==============================] - 22s 2s/step - loss: 9.0351 - mse_known: 9.3140e-07 - mse_unknown: 0.3282


Filter Coefficients: [9.5388, 8.442761]


###########################################################################################
################################### Sensing Ratio: 40.0% ###################################
###########################################################################################


Epoch 1/30
10/10 [==============================] - 23s 2s/step - loss: 9.2678 - mse_known: 8.2459e-07 - mse_unknown: 0.4445 - lr: 0.0040
Epoch 2/30
10/10 [==============================] - 23s 2s/step - loss: 9.1858 - mse_known: 8.1762e-07 - mse_unknown: 0.4440 - lr: 0.0040
Epoch 3/30
10/10 [==============================] - 23s 2s/step - loss: 9.1045 - mse_known: 8.1054e-07 - mse_unknown: 0.4434 - lr: 0.0040
Epoch 4/30
10/10 [==============================] - 20s 2s/step - loss: 9.0240 - mse_known: 8.0366e-07 - mse_unknown: 0.4429 - lr: 0.0040
Epoch 5/30
10/10 [==============================] - 20s 2s/step - loss: 8.9441 - mse_known: 7.9671e-07 - mse_unknown: 0.4423 -


Filter Coefficients: [9.542016, 8.449524]


###########################################################################################
################################### Sensing Ratio: 50.0% ###################################
###########################################################################################


Epoch 1/30
10/10 [==============================] - 20s 2s/step - loss: 9.1977 - mse_known: 7.0398e-07 - mse_unknown: 0.6258 - lr: 0.0040
Epoch 2/30
10/10 [==============================] - 21s 2s/step - loss: 9.1163 - mse_known: 6.9814e-07 - mse_unknown: 0.6249 - lr: 0.0040
Epoch 3/30
10/10 [==============================] - 26s 2s/step - loss: 9.0355 - mse_known: 6.9231e-07 - mse_unknown: 0.6239 - lr: 0.0040
Epoch 4/30
10/10 [==============================] - 23s 2s/step - loss: 8.9555 - mse_known: 6.8659e-07 - mse_unknown: 0.6229 - lr: 0.0040
Epoch 5/30
10/10 [==============================] - 21s 2s/step - loss: 8.8762 - mse_known: 6.8096e-07 - mse_unknown: 0.6219

In [17]:
for i, v in enumerate(all_vars):
    print(f"For {int(100 * sensing_ratio[i])}% sensing ratio: ", v)

For 1% sensing ratio:  [9.559109, 8.480312]
For 5% sensing ratio:  [9.559249, 8.480834]
For 10% sensing ratio:  [9.559454, 8.481509]
For 20% sensing ratio:  [9.559808, 8.482651]
For 30% sensing ratio:  [9.5388, 8.442761]
For 40% sensing ratio:  [9.542016, 8.449524]
For 50% sensing ratio:  [9.545073, 8.453857]


In [18]:
# np.save('./Outputs/loss_model.npy', loss_model)
# np.save('./Outputs/learned_graph.npy', learned_graph)
# np.save('./Outputs/mse_known.npy', mse_known_list)
np.save('./Outputs/mse_unknown_v1.npy', mse_unknown_list)